In [4]:
from Models.models import LLMModel
from langchain_core.messages import (HumanMessage)
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field, validator
import json

## Create a simple callable model

In [9]:


model = LLMModel(
    provider="openai_api",
    model="gpt-4-1106-preview",
    output_schema=None, 
    )


# YOU CAN EITHER PASS A LIST OF LANGCHAIN CORE MESSAGES OR A DICTIONARY WITH KEY `MESSAGES`
response = model(
    [HumanMessage(content="Who is the president of Nigeria")]
    )
print(response)

print("--------------------------------")

response = model(
    {
        "messages": [HumanMessage(content="Who is the president of Nigeria")]
    }
    )
print(response)


As of my last update in 2023, the President of Nigeria is Muhammadu Buhari. He has been in office since May 29, 2015, after winning the presidential election of that year. Buhari was re-elected for a second term in the 2019 presidential elections. However, political situations can change, so for the most current information, it would be best to check the latest news or official sources.
--------------------------------
As of my last update in early 2023, the President of Nigeria is Muhammadu Buhari. However, please note that Nigeria was due to hold presidential elections in February 2023. Depending on the current date and any developments since my last update, there might be a new president if the elections have taken place and a new president has been inaugurated. For the most accurate and current information, please check the latest news or official announcements.


### Adding Output Schema and prompt template


In [10]:
class Joke(BaseModel):
    """Joke model"""
    setup: str = Field(..., description="setup for the joke")
    punchline: str = Field(..., description="punchline for the joke")


prompt_template = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "Given this topic={topic}, generate a joke."
                ),
                (
                    "human",
                    "Also, make your output all upper case LIKE THIS."
                )
               
            ]
        )



model1 = LLMModel(
    provider="fireworks_api",
    model="llama-v3-70b-instruct",
    output_schema=Joke,
    try_to_parse = True, # model will return JSON object based on the output schema
    config = {"retry": 2, "retry_with_history": False}, #if model fails to output parsable response, it retries 2 time without providing failed response as feedback
    prompt_template=prompt_template
    )


inputs = {"topic": "dad jokes"}
response = model1(inputs)
    
print(response)
print(type(response))

{'SETUP': 'WHY DID THE SCARECROW WIN AN AWARD?', 'PUNCHLINE': 'BECAUSE HE WAS OUTSTANDING IN HIS FIELD!'}
<class 'dict'>


## Using as an Evaluator

The Models.schemas module comes with some pre-defined pydantic schemas:
1. **FeedbackISC**: This can be used as an output schema for evaluation to get `Issues`, `Score` and `Comments`
2. **FeedbackBasic**: This can be used as an output schema to get a a dictionary with response (Default value when `try_to_parse` is set to true)
3. **QualityAspect**: This is used to define the Evaluation Quality aspect with an `AspectorEvaluatorInputSchema` schema
4. **AspectorRole**: This is used to specify if the evaluator should judge only the `USER`, `ASSISTANT` or both (`USER_AND_ASSISTANT`)

In [13]:
from Models.schemas import AspectorEvaluatorInputSchema, FeedbackISC, FeedbackBasic,  QualityAspect, AspectorRole


#TO TAKE ADVANTAGE OF THE SPECIAL EVALUATOR PROMPT, SET as_evaluator to True
evaluator = LLMModel(
    provider="fireworks_api",
    model="llama-v3-70b-instruct",
    output_schema=FeedbackISC, 
    input_schema=AspectorEvaluatorInputSchema, 
    name="aspect_evaluator",
    as_evaluator = True,
    try_to_parse=True
)

#Conversation to be judged by the evaluator
convo = [
    "user: Hello, are you doing today?",
    "assistant: I'm doing well, thank you for asking! How can I assist you today?",
    "user: I was hoping you could provide some feedback, on a short essay. I wrote. Would you be able to take a look and let me know your thoughts on the grammar and structure?",
    "assistant: Absolutely, I'd be happy to review, your essay and provide feedback on the grammar and overall structure. Please go ahead and send over the essay whenever you're ready, and I'll take a look. Let me know if there are any specific areas you'd like me to focus on in addition to the grammar and structure."
    ]


#Define the aspect to be judge
quality_aspect = QualityAspect(
    name="grammar",
    instruction="The quality of the grammar including how well it is structured and punctuated"
)


# Set the evaluation task (model input)
evaluation_task = AspectorEvaluatorInputSchema(
    quality_aspect=quality_aspect,
    role=AspectorRole.USER_AND_ASSISTANT,
    conversation=convo,
    metadata = {}

)


# Perform evaluation
evaluation_result = evaluator(evaluation_task)
print(evaluation_result)
print("--------------------------------")
print(type(evaluation_result))


{'issues': ["assistant's sentence is a bit long and could be broken up for better clarity."], 'score': 4, 'comment': "The grammar and structure of both user and assistant responses are generally good, with proper use of punctuation and clear sentence structure. However, the assistant's response could be improved by breaking up a long sentence for better clarity."}
--------------------------------
<class 'dict'>


### Adding Extra Model configuration (Parameters and Model Kwargs)



In [28]:
model = LLMModel(
    provider="openai_api",
    model="gpt-4-1106-preview",
    output_schema=FeedbackBasic,
    prompt_template=ChatPromptTemplate.from_messages([("system",
                                         "You are a funny virtual assistant")]), #add extra personality configuration
    try_to_parse = True, 
    config = {"retry": 2, 
              "retry_with_history": True,
              "params": {'temperature':0.9}, # Extra model parameters goes here
              #"model_kwargs": {}  # Extra kwargs can be added here
              }
    )


messages=[HumanMessage(content="What do you call a dog with 3 legs?"),]
response = model(messages)
print(response)

print("--------------------------------")

messages=[HumanMessage(content="What about  a dog with 2 legs?"),]
response = model(messages)
print(response)

print("--------------------------------")

messages=[HumanMessage(content="1 leg?"),]
response = model(messages)
print(response)

{'response': "A three-legged dog. But remember, it doesn't matter how many legs a dog has; it's still a dog-gone good friend!"}
--------------------------------
{'response': "You might call it a 'bi-pawed' friend, but no matter the number of legs, it's still paws-itively lovable!"}
--------------------------------
{'response': "A one-legged dog? That's a 'uni-pawed' pal – still just as fetching as any other!"}


## CHAT HISTORY AND TOKEN COUNT
1. use `model.chat_history`  to retrieve chat history without system prompt (or prompt template)
2. use `model.get_chat_history()` to retrieve chat history with system prompt (or prompt template)
3. use `model.chat_history_untouched` to retrieve chat history with system prompt and failed responses (failed parsed responses)
4. use `model.get_total_tokens()` to retrieve  both token count and content used to retrieve the count

In [25]:
# without system prompt
model.chat_history

[HumanMessage(content='What do you call a dog with 3 legs?'),
 AIMessage(content='A dog with three legs can be lovingly referred to as a "tripod" or a "tricycle," but no matter what you call them, they\'re often just as capable and adorable as their four-legged pals. And remember, while it\'s okay to share a chuckle about clever names, it\'s important to always show compassion and kindness to our differently-abled canine friends. 🐾', response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 30, 'total_tokens': 111}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7fbcc817-d991-4048-a1e2-3211fd5cc510-0'),
 HumanMessage(content='What about  a dog with 2 legs?'),
 AIMessage(content='A dog with two legs may not have a specific funny name, but I\'ve heard some endearingly be called "bipods." However, no matter what their condition, they tend to be incredibly resilient and adaptive. Some of these remarkabl

In [26]:
#with system prompt
model.get_chat_history()

[SystemMessage(content='\nYou are a funny virtual assistant\n'),
 HumanMessage(content='What do you call a dog with 3 legs?'),
 AIMessage(content='A dog with three legs can be lovingly referred to as a "tripod" or a "tricycle," but no matter what you call them, they\'re often just as capable and adorable as their four-legged pals. And remember, while it\'s okay to share a chuckle about clever names, it\'s important to always show compassion and kindness to our differently-abled canine friends. 🐾', response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 30, 'total_tokens': 111}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7fbcc817-d991-4048-a1e2-3211fd5cc510-0'),
 HumanMessage(content='What about  a dog with 2 legs?'),
 AIMessage(content='A dog with two legs may not have a specific funny name, but I\'ve heard some endearingly be called "bipods." However, no matter what their condition, they tend

In [27]:
#with failed parses
model.chat_history_untouched

[HumanMessage(content='What do you call a dog with 3 legs?'),
 AIMessage(content='A dog with three legs can be lovingly referred to as a "tripod" or a "tricycle," but no matter what you call them, they\'re often just as capable and adorable as their four-legged pals. And remember, while it\'s okay to share a chuckle about clever names, it\'s important to always show compassion and kindness to our differently-abled canine friends. 🐾', response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 30, 'total_tokens': 111}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7fbcc817-d991-4048-a1e2-3211fd5cc510-0'),
 HumanMessage(content='What about  a dog with 2 legs?'),
 AIMessage(content='A dog with two legs may not have a specific funny name, but I\'ve heard some endearingly be called "bipods." However, no matter what their condition, they tend to be incredibly resilient and adaptive. Some of these remarkabl

In [29]:
#Get Tokens
tokens, tokens_string = model.get_total_tokens()

DEBUG: ...API token count


In [30]:
tokens

{'in': 715, 'out': 105}

In [32]:
tokens_string["in"]

['\nYou are a funny virtual assistant\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"response": {"description": "The response string returned by the LLM", "title": "Response", "type": "string"}}, "required": ["response"]}\n```What do you call a dog with 3 legs?',
 '\nYou are a funny virtual assistant\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\

In [33]:
tokens_string["out"]

['{\n  "response": "A three-legged dog. But remember, it doesn\'t matter how many legs a dog has; it\'s still a dog-gone good friend!"\n}',
 '{\n  "response": "You might call it a \'bi-pawed\' friend, but no matter the number of legs, it\'s still paws-itively lovable!"\n}',
 '{\n  "response": "A one-legged dog? That\'s a \'uni-pawed\' pal – still just as fetching as any other!"\n}']

###  Turn off chat History

This will save token cost since every new message is passed to the model without the chat history included


In [37]:
model = LLMModel(
    provider="openai_api",
    model="gpt-4-1106-preview",
    use_history = False,     
    )


messages=[HumanMessage(content="What do you call a dog with 3 legs?"),]
response = model(messages)
print(response)

print("--------------------------------")

messages=[HumanMessage(content="What about  a dog with 2 legs?"),]
response = model(messages)
print(response)



{'response': "A dog with three legs can simply be called a 'tripod' as a nickname, but it is still just a dog and can be called by its given name."}
--------------------------------
{'response': 'A dog with two legs would require special care and accommodations to ensure its well-being. Depending on which legs are affected, the dog may need a wheelchair, prosthetics, or other mobility aids to move around. Moreover, the dog would likely need physical therapy and a safe environment to prevent injuries and to enhance its quality of life.'}


In [38]:
model.chat_history #You will get and empty history here

[]